In [ ]:
from run_generation import sample_sequence
from yt_encoder import YTEncoder
from transformers import GPT2LMHeadModel
import regex as re
import youtokentome as yttm
import logging
import torch
import os

logger = logging.getLogger('test_gener')

In [ ]:
from apex import amp

In [ ]:
import torch

torch.cuda.is_available()

In [ ]:
model_path = '/workspace/pelevin_m'

In [ ]:
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = YTEncoder.from_pretrained(model_path)

In [ ]:
device = 'cuda'
model.to(device)
model.eval()

In [ ]:
def get_sample(model, prompt, length:int, num_samples:int, allow_linebreak:bool):
    logger.info("*" * 200)
    logger.info(prompt)
   
    filter_n = tokenizer.encode('\n')[-1:]
    filter_single = [1] + tokenizer.encode('[')[-1:] + tokenizer.encode('(')[-1:]
    filter_single += [] if allow_linebreak else filter_n

    context_tokens = tokenizer.encode(prompt)
    out = sample_sequence(
        model=model,
        context=context_tokens,
        length=length,
        temperature=1,
        top_k=0,
        top_p=0.9,
        device=device,
        filter_single=filter_single,
        filter_double=filter_n,
        num_samples=num_samples,
    ).to('cpu')

    prompt = tokenizer.decode(context_tokens)
    len_prompt = len(prompt)
   
    replies = [out[item, :].tolist() for item in range(len(out))]
    text = [tokenizer.decode(item)[len_prompt:] for item in replies]
    reg_text = [re.match(r'[\w\W]*[\.!?]\n', item) for item in text]
    reg_text2 = [re.match(r'[\w\W]*[\.!?]', item) for item in text]
    result = [reg_item[0] if reg_item else reg_item2[0] if reg_item2 else item for reg_item, reg_item2, item in zip(reg_text, reg_text2, text)]
    logger.info("=" * 200)
    logger.info(result)
    return result

In [ ]:
get_sample(model, 'хватит нести чушь', 30, 3, False)